## Import the Pandas library

import pandas as pd
import os

## EXTRACTION

In [10]:


# Create a reference the CSV file desired
monstercsv = os.path.join("Resources", "monster_com-job.csv")
dicecsv = os.path.join("Resources", "Dice_US_jobs.csv")

# Read the CSV into a Pandas DataFrame
monster_df = pd.read_csv(monstercsv)
dice_df = pd.read_csv(dicecsv,  encoding="ISO-8859-1")

# show tatal raw for each data frame
print("totoal job post in    dice: {dice_df.count()}")
print("totoal job post in monster: {monster_df.count()}")


totoal job post in    dice: {dice_df.count()}
totoal job post in monster: {monster_df.count()}


In [16]:
# examine top 5 rows for monster data frame
monster_df.head()

,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


In [17]:
# examine top 5 rows for dice data frame
dice_df.head()

,country_code,date_added,job_board,job_description,job_title,job_type,location,organization,page_url,phone_number,salary,sector
0,US,11/11/2016,https://www.dice.com,"Minimum Required Skills:EDI, TrustedLink, AS2,...",EDI Analyst,"Full Time, Full-time, Employee","Stamford, CT",CyberCoders,https://www.dice.com/jobs/detail/EDI-Analyst-%...,NaN,NaN,"EDI, TrustedLink, AS2, VAN - EDI, TrustedLink,..."
1,US,11/11/2016,https://www.dice.com,"InformaticaåÊ/ ETL DeveloperSt, Petersburg, FL...",Informatica ETL Developer,"Full Time, Full Time","St Petersburg, FL",TrustMinds,https://www.dice.com/jobs/detail/Informatica-E...,NaN,NaN,ETL Informatica B2B Data Exchange Netezza Orac...
2,US,11/11/2016,https://www.dice.com,pmayekar@kanandcorp.com 512-697-8897Sunnyvale ...,Angular developer,"Full Time, Contract Corp-To-Corp, Contract Ind...","Sunnyvale, CA",K Anand Corporation,https://www.dice.com/jobs/detail/Angular-devel...,NaN,NaN,Angular
3,US,11/12/2016,https://www.dice.com,This nationally recognized Microsoft Gold Part...,"Microsoft Dynamics AX, Project Manager",Full Time,"Toronto, Canada, ON",Nigel Frank International,https://www.dice.com/jobs/detail/Microsoft-Dyn...,NaN,NaN,"Microsoft Dynamics AX, Project Manager - Toron..."
4,US,11/11/2016,https://www.dice.com,"Minimum Required Skills:C#, ASP.NET, SQL, Java...",Software Developer,"Full Time, Full-time, Employee","Stamford, CT",CyberCoders,https://www.dice.com/jobs/detail/Software-Deve...,NaN,NaN,"C#, ASP.NET, SQL, JavaScript, MVC - C#, ASP.NE..."


## Transform monster DataFrame

## Transform dice DataFrame

## Create database connection

## Load data frame into database